In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import gudhi as gd
from open3d import io, visualization
from open3d import geometry, utility
import networkx as nx
import random
import numpy as np
import open3d as o3d


In [9]:

def random_color_generator():
    r = random.random()
    g = random.random()
    b = random.random()
    return (r, g, b)


def split_cloud(bag, t, visualize=False):
    
    filename = f"persistence_threshold_points\{bag}-{t}.txt"
    
    if os.path.isfile(filename) and not visualize: return

    # Path
    pathToData = os.path.join(os.getcwd(), 'data', 'bags', bag)

    # Read Data
    pointCloud = io.read_point_cloud(os.path.join(pathToData, 'cloud_final.ply')) # Read point cloud
    pointCloudArray = np.asarray(pointCloud.points)

    if not os.path.isfile(filename):

        # # Create alpha complex
        alphaComplex = gd.AlphaComplex(points = pointCloudArray)

        # # Create simplex tree object
        simplexTree = alphaComplex.create_simplex_tree()
        simplexTree.prune_above_filtration(t)

        points = []

        # Compute adjacency list from skeleton.
        for sk_value in simplexTree.get_skeleton(1):
            i = sk_value[0][0]
            while len(points) != i+1:
                points.append([])
            if sk_value[1] == 0: continue

            v = sk_value[0][1]

            points[i].append(v)

        # Add unidirectional edges
        for i in range(len(points)-1, 0, -1):
            for j in points[i]:
                points[j].append(i)
            points[i].insert(0, i)

        
        # Save adjacencylist to file

        # with open(filename,'r+') as file:
        #     file.truncate(0)

        f = open(filename, "a") 
        for p in points:
            for d in p:
                f.write(f"{d} ")
            f.write("\n")
        f.close()

    if visualize:

        # Compute disconnected components
        graph = nx.read_adjlist(filename, nodetype=int)
        subgraphs = nx.connected_components(graph)

        # Retrieve all points from original cloud
        clouds = []
        for sg in subgraphs:

            segmentedCloud = [pointCloudArray[i] for i in sg]

            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(segmentedCloud)
            pcd.paint_uniform_color(random_color_generator())

            clouds.append(pcd)


        visualization.draw_geometries(clouds)


split_cloud("bag_0", 0.0001, visualize=True)
split_cloud("bag_1", 0.0001)
split_cloud("bag_2", 0.0001)
split_cloud("bag_3", 0.0001)
split_cloud("bag_4", 0.0001)
split_cloud("bag_5", 0.0001)
